<a href="https://colab.research.google.com/github/rivervaladao/FIAP-POS-TECH/blob/master/IA_PLN/aula04/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/Word2Vec/cbow_s300.zip"

In [ ]:
import pandas as pd
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('cbow_s300.txt')


In [1]:
# Extract words and their corresponding vectors
words = model.index_to_key  # List of words
vectors = [model[word] for word in words]

NameError: name 'model' is not defined

In [ ]:
df = pd.DataFrame(vectors, index=words)


In [ ]:
df.head(5)

In [ ]:
model.get_vector("china")

In [ ]:
model.most_similar("china")

In [ ]:
model.most_similar(positive=["brasil", "uruguai"])

In [ ]:
#carro -> carros : foguete -> foguetes
#carros + foguete - carro = foguetes

model.most_similar(positive=["carros", "foguete"], negative=["carro"])

**Vetorização**

In [ ]:
artigo_treino.title[12]

In [ ]:
import nltk
import string
nltk.download('punkt')

def tokenizador(texto):
  texto = texto.lower()
  lista_alfanumerico = []

  for token_valido in nltk.word_tokenize(texto):
    if token_valido in string.punctuation: continue
    lista_alfanumerico.append(token_valido)

  return lista_alfanumerico

In [ ]:
tokenizador("Texto com, pontuação.")

In [ ]:
import numpy as np

def combinacao_de_vetores_por_soma(palavras_numeros):
  vetor_resultante = np.zeros(300)
  for pn in palavras_numeros:
    try:
      vetor_resultante =+ modelo.get_vector(pn)
    except KeyError:
      if pn.isnumeric():
        pn = "0"*len(pn)
        vetor_resultante =+ modelo.get_vector(pn)
      else:
        vetor_resultante =+ modelo.get_vector('unknown')
  return vetor_resultante

In [ ]:
palavras_numeros = tokenizador("texto fiaps")
vetor_texto = combinacao_de_vetores_por_soma(palavras_numeros)
print(len(vetor_texto))
print(vetor_texto)

In [ ]:
def matriz_vetores(textos):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras_numeros = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras_numeros)

  return matriz

In [ ]:
matriz_vetores_treino = matriz_vetores(artigo_treino.title)
matriz_vetores_teste = matriz_vetores(artigo_teste.title)
print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(max_iter = 200)
LR.fit(matriz_vetores_treino, artigo_treino.category)

In [ ]:
LR.score(matriz_vetores_teste, artigo_teste.category)

In [ ]:
artigo_teste.category.unique()

In [ ]:
from sklearn.metrics import classification_report

label_prevista = LR.predict(matriz_vetores_teste)
CR = classification_report(artigo_teste.category, label_prevista)
print(CR)

In [ ]:
from sklearn.dummy import DummyClassifier

DC = DummyClassifier()
DC.fit(matriz_vetores_treino, artigo_treino.category)
label_previsao_dc = DC.predict(matriz_vetores_teste)
CR_dummy = classification_report(artigo_teste.category, label_previsao_dc)
print(CR_dummy)

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/Word2Vec/skip_s300.zip"

In [ ]:
modelo_skipgram = KeyedVectors.load_word2vec_format("skip_s300.txt")

In [ ]:
def combinacao_de_vetores_por_soma_skipgram(palavras_numeros):
  vetor_resultante = np.zeros(300)
  for pn in palavras_numeros:
    try:
      vetor_resultante =+ modelo_skipgram.get_vector(pn)
    except KeyError:
      if pn.isnumeric():
        pn = "0"*len(pn)
        vetor_resultante =+ modelo_skipgram.get_vector(pn)
      else:
        vetor_resultante =+ modelo_skipgram.get_vector('unknown')
  return vetor_resultante

In [ ]:
def matriz_vetores_skipgram(textos):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras_numeros = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma_skipgram(palavras_numeros)

  return matriz

In [ ]:
matriz_vetores_treino_skipgram = matriz_vetores_skipgram(artigo_treino.title)
matriz_vetores_teste_skipgram = matriz_vetores_skipgram(artigo_teste.title)

In [ ]:
LR_skipgram = LogisticRegression(max_iter = 200)
LR_skipgram.fit(matriz_vetores_treino_skipgram, artigo_treino.category)
label_previsao_skipgram = LR_skipgram.predict(matriz_vetores_teste_skipgram)
CR_skipgram = classification_report(artigo_teste.category, label_previsao_skipgram)
print(CR_skipgram)